## Optimización de Hiperparámetros

El siguiente ejemplo implementa la solución a problema básico de clasificación usando una red neuronal (MLPClassifier), entrenada con el conjunto de datos digits (dígitos manuscritos), y con optimización bayesiana de hiperparámetros usando Optuna.

### Importar librerías

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt

try:
    # Librería de optimización bayesiana para ajuste de hiperparámetros
    import optuna
    print("Librería 'optuna' instalada y cargada correctamente")
    print(optuna.__version__)
except:
    print("Librería 'optuna' no instalada, se procederá con la instalación")
    !pip install scikit-learn optuna
    # Librería de optimización bayesiana para ajuste de hiperparámetros
    import optuna
    print(optuna.__version__)

# Conjunto de datos de dígitos manuscritos incluido en scikit-learn
from sklearn.datasets import load_digits

# Funciones para dividir los datos y realizar validación cruzada
from sklearn.model_selection import train_test_split, cross_val_score

# Clasificador de perceptrón multicapa (red neuronal)
from sklearn.neural_network import MLPClassifier

# Normalizador de características (media = 0, varianza = 1)
from sklearn.preprocessing import StandardScaler

# Pipelines o tuberías (flujo de procesamiento) de pasos de preprocesamiento y modelado
# Garantiza que primero se transforman los datos (fit/transform) y luego se entrene o evalúe el modelo final (fit/predict).
from sklearn.pipeline import make_pipeline

# Métrica para evaluar la precisión del modelo (proporción de aciertos)
from sklearn.metrics import accuracy_score

# Ignorar las advertencias
import warnings
warnings.filterwarnings("ignore")

### Cargar datos y dividirlos en entrenamiento y pruebas

In [ ]:
# Carga el conjunto de datos de dígitos manuscritos (8x8 píxeles)
digits = load_digits()

# Separa las características (X) y las etiquetas (y) del conjunto de datos
X, y = digits.data, digits.target

# Divide el conjunto de datos en entrenamiento (80%) y prueba (20%)
# El parámetro random_state asegura que la división sea reproducible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Descripción del dataset

In [ ]:
print(digits.DESCR)

### Convertir a DataFrame para visualizar el conjunto de datos

In [ ]:
df = pd.DataFrame(digits.data, columns=digits.feature_names)

# Agregar la columna de etiquetas
df['target'] = digits.target

# Mostrar las primeras 5 filas
df.head()

### Dimensiones del dataset

In [ ]:
print(df.shape)
print('Cantidad de registros:', df.shape[0])
print('Cantidad de variables:', df.shape[1])

### Desplegar la imagen del número ubicado en la fila 0

In [ ]:
# Extraer el primer ejemplo
# reshape convierte el vector de 64 elementos en una matriz de 2 dimensiones 8×8
number = X[0].reshape(8, 8)

# Graficar la imagen
plt.figure(figsize=(2, 2))
plt.imshow(number, cmap=plt.cm.gray_r)  # cmap puede ser 'binary', 'gray' o 'gray_r'
plt.title(f"Etiqueta real: {y[0]}")  # Mostrar la etiqueta del dígito
plt.tight_layout()
plt.show()

### Función objetivo para Optuna

In [ ]:
def objective(trial):
    # Definición del espacio de búsqueda

    # Definir el número y tamaño de las capas ocultas de la red neuronal
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(64,), (100,), (64, 64), (100, 50)])

    # Función de activación para las neuronas de la red
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])

    # Algoritmo de optimización (solver) usado para entrenar la red
    solver = trial.suggest_categorical('solver', ['adam', 'sgd'])

    # Parámetro de regularización L2 (penalización de pesos grandes)
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)

    # Tasa de aprendizaje inicial para el optimizador
    learning_rate_init = trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)

    # Crear un pipeline (flujo de procesamiento) que incluye estandarización de datos
    # y un clasificador MLP (red neuronal multicapa)
    clf = make_pipeline(

        # Paso 1: Estandarizar los datos con media=0 y desviación estándar=1.
        # Esto es importante porque las redes neuronales son sensibles a la escala de las variables
        StandardScaler(),

        # Paso 2: Entrenar un clasificador MLP (red neuronal multicapa) con los hiperparámetros seleccionados
        MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,   # Arquitectura de la red (número de neuronas en cada capa oculta).
            activation=activation,                   # Función de activación ('relu', 'tanh' o 'logistic').
            solver=solver,                           # Algoritmo de optimización ('adam' o 'sgd').
            alpha=alpha,                             # Parámetro de regularización L2 para evitar sobreajuste.
            learning_rate_init=learning_rate_init,   # Tasa de aprendizaje inicial.
            max_iter=100,                            # Número máximo de iteraciones (épocas) de entrenamiento.
            random_state=42                          # Semilla para reproducibilidad de resultados.
        )
    )

    # Validación cruzada
    score = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')

    return score.mean()

### Ejecutar la optimización

In [ ]:
%%time

# Crear un estudio de Optuna para optimización de hiperparámetros.
# direction='maximize' → indica que queremos maximizar la métrica objetivo
# (ejemplo: accuracy, AUC, F1). Si quisiéramos minimizar (ej. RMSE, error),
# se usaría direction='minimize'.
study = optuna.create_study(direction='maximize')

# Ejecutar la optimización del estudio.
#   - objective   → función que entrena y evalúa el modelo, devolviendo la métrica que se desea optimizar.
#   - n_trials=10 → número de experimentos (combinaciones de hiperparámetros) que Optuna va a probar.
study.optimize(objective, n_trials=10)

# Mostrar mejores hiperparámetros
print("\nMejores hiperparámetros encontrados:")
print(study.best_params)

### Evaluar el mejor modelo en el conjunto de prueba

In [ ]:
%%time

# Obtener los mejores hiperparámetros encontrados por Optuna
best_params = study.best_params

# Crear un pipeline con dos pasos:
# 1. Estandarización de datos (StandardScaler)
# 2. Red neuronal multicapa (MLPClassifier) con los mejores hiperparámetros
best_model = make_pipeline(
    StandardScaler(),   # Escalar los datos (media=0, varianza=1) antes de entrenar

    MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],   # Arquitectura óptima (número y tamaño de capas ocultas)
        activation=best_params['activation'],                   # Función de activación óptima (relu, tanh o logistic)
        solver=best_params['solver'],                           # Algoritmo de optimización óptimo (adam o sgd)
        alpha=best_params['alpha'],                             # Regularización L2 óptima (para evitar sobreajuste)
        learning_rate_init=best_params['learning_rate_init'],   # Tasa de aprendizaje inicial óptima
        max_iter=100,                                           # Número máximo de iteraciones (épocas) de entrenamiento
        random_state=42                                         # Semilla para reproducibilidad
    )
)

# Entrenar el modelo con los datos de entrenamiento
best_model.fit(X_train, y_train)

# Predecir las etiquetas en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular la precisión comparando predicciones con valores reales
acc = accuracy_score(y_test, y_pred)

# Mostrar la precisión en el conjunto de prueba con 4 decimales
print(f"Precisión en el conjunto de prueba: {acc:.4f}")

### Resumen de la evaluación

In [ ]:
# Imprimir el número total de elementos en el conjunto de prueba
print("Elementos de prueba: {}".format(y_test.shape[0]))

# Contar cuántas predicciones fueron incorrectas (comparando y_test vs y_pred)
print("Errores identificados: {}".format((y_test != y_pred).sum()))

# Calcular el porcentaje de error:
#   - Número de errores dividido entre total de elementos de prueba
#   - Multiplicado por 100 para expresarlo en porcentaje
porcentaje_error = ((y_test != y_pred).sum() * 100) / y_test.shape[0]

# Mostrar el porcentaje de error con formato de texto
print("Porcentaje de error: {} %".format(porcentaje_error))

### Errores en las predicciones del conjunto de datos de prueba

In [ ]:
# Seleccionar las muestras del conjunto de prueba en las que el modelo se equivocó.
errores = X_test[y_test != y_pred]

# Extraer las etiquetas reales correspondientes a esas muestras mal clasificadas
real_labels = y_test[y_test != y_pred]
print("Etiquetas reales    :", real_labels)

# Extraer las etiquetas que el modelo predijo para esas mismas muestras
predicted_labels = y_pred[y_test != y_pred]
print("Etiquetas estimadas :", predicted_labels)

# Imprimir cuántos errores cometió el modelo en el conjunto de prueba
print("Cantidad de errores:", len(errores))

### Graficar las primeras 5 filas de errores

In [ ]:
k = 0   # Contador para limitar cuántos bloques de imágenes mostrar

# Recorrer los errores en bloques de 5 muestras cada uno
# math.ceil() permite "redondear hacia arriba" (ceil = techo), ademas
# garantiza que no se pierda ningún error aunque el número total no sea múltiplo exacto de 5
for j in range(math.ceil(errores.shape[0] / 5)):

    # Crear una figura de 10x10 pulgadas para cada bloque
    plt.figure(figsize=(10,10))

    # Sub-bucle para mostrar 5 imágenes por fila
    for i in range(5):

        idx = 5*j + i
        if idx >= errores.shape[0]:   # Evitar pasarse si no hay más errores
            break

        # Definir la posición del subplot (1 fila, 5 columnas, posición i+1)
        plt.subplot(1, 5, i + 1)

        # Eliminar marcas en los ejes x e y
        plt.xticks([])
        plt.yticks([])

        # Quitar la cuadrícula
        plt.grid(False)

        # Recuperar la imagen del error correspondiente (vector plano de 64 → 8x8)
        number = errores[5*j + i].reshape(8, 8)

        # Mostrar la imagen en blanco y negro (binary = blanco/negro invertido)
        plt.imshow(number, cmap=plt.cm.binary)

        # Etiquetar con la clase real y la predicción del modelo
        # El bucle externo (for j in range(...)) avanza en bloques de 5 errores cada vez.
        # El bucle interno (for i in range(5)) recorre los 5 elementos dentro de ese bloque.
        # Entonces, el índice total dentro del arreglo de errores no es solo i ni solo j, sino una combinación de ambos:
        #   índice total=(nuˊmero_de_bloque × tamaño_del_bloque) + índice_dentro_del_bloque
        plt.xlabel("Real: {} Predicción: {}".format(real_labels[5*j+i], predicted_labels[5*j+i]))

    # Mostrar la figura completa con las 5 imágenes
    plt.show()

    # Aumentar el contador de bloques
    k += 1

    # Romper el bucle si ya se mostraron 5 bloques (máximo 25 imágenes)
    if k == 5:
        break

### ¿Qué hace este ejemplo?

* Usa el conjunto digits (1797 imágenes de dígitos manuscritos).
* Crea un MLPClassifier con diferentes combinaciones de hiperparámetros (capas ocultas, activación, alpha, tasa de aprendizaje...).
* Usa Optuna para optimizar esos hiperparámetros mediante validación cruzada.
* Evalúa el modelo final con los mejores parámetros sobre el conjunto de prueba.